In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import talib


# Define the Linear Regression model
class LinearRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0

        for _ in range(self.n_iterations):
            # Calculate predictions
            y_pred = np.dot(X, self.weights) + self.bias

            # Compute gradients
            dw = (1 / num_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / num_samples) * np.sum(y_pred - y)

            # Update weights and bias
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias
    

    # Data loading
def data_loader(filepath):
    data = pd.read_csv(filepath)
    data['Time'] = pd.to_datetime(data['Time'],format='%Y-%m-%d %H:%M:%S')
    data.set_index('Time', inplace=True)

    return data


# Feature Engineering

def multivariateFeatureEngineering(filepath):
    data = pd.read_csv(filepath)

    data['Time'] = pd.to_datetime(data['Time'],format='%Y-%m-%d %H:%M:%S')
    
    '''
        Feature Egineering - Added features: MA, EMA, MACD, RSI
    '''
    #Trend following Indicators:

    #SMA - identofy long term trend
    data['50_sma'] = data['Close'].rolling(window=50).mean() 
    data['200_sma'] = data['Close'].rolling(window=200).mean() 

    #EMA - trend analysis: more weight applied to recent points
    data['50_ema'] = data['Close'].ewm(span=50, adjust=False).mean()
    data['100_ema'] = data['Close'].ewm(span=100, adjust=False).mean()

    #MACD
    data['12_ema'] = data['Close'].ewm(span=12, adjust=False).mean()
    data['26_ema'] = data['Close'].ewm(span=26, adjust=False).mean()
    data['MACD_line'] = data['12_ema']-data['26_ema'] # calculate the MACD line
    data['Signal_line'] = data['MACD_line'].ewm(span=9, adjust=False).mean() # 9-preiod ema signal calculated from the Macdline
    data['MACD_histogram'] = data['MACD_line'] - data['Signal_line']

    #ADX
    # Calculate ADX using TA-Lib (14-period by default)
    data['ADX'] = talib.ADX(data['High'], data['Low'], data['Close'], timeperiod=14)

    #Momentum indicators:

    #RSI - 14-period
    data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
    
    #Stochastic Oscillator
    data['stoch_k'], data['stoch_d'] = talib.STOCH(data['High'], data['Low'], data['Close'], 
                                                fastk_period=14, slowk_period=3, slowd_period=3)

    #Williams %R - Default period is 14
    data['Williams_R'] = talib.WILLR(data['High'], data['Low'], data['Close'], timeperiod=14)

    #Volatility indicators#:

    #Bollinger Bands - Default period is 20 for moving average and standard deviation (can be adjusted)
    data['upper_band'], data['middle_band'], data['lower_band'] = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

    #ATR -Default period for ATR is 14
    data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)

   
    data = data.dropna() # drop rows that have NA

    return data


# reshape an input data into n_samples x timesteps x n_features.
# NOTE: data should be standardized

def multivariateFeatureLag(data, n_past, n_future, target_column=1):
    features = []
    response = []

    for i in range(n_past, len(data) - n_future + 1):
        # Extract the past n_past time steps as features
        features.append(data[i - n_past:i, :])  # Take all columns as features

        response.append(data[i + n_future - 1, target_column])  # Extract the target column for response

    # Move the conversion to NumPy arrays outside the loop
    features = np.array(features)  # Convert to NumPy array after the loop finishes
    response = np.array(response)  # Convert to NumPy array after the loop finishes

    features_flat = features.reshape(features.shape[0], -1) # make shape 2d

    return features_flat, response



import itertools
import logging
import talib
import csv

# Define the file and the header
csv_file = 'hyperparameter_results.csv'
header = ['Learning Rate', 'Lookback Window', 'Features', 'Test MSE', 'Test MAE','Test MBE', 'Test R2']


features = ['Open', 'High', 'Low', 'Volume', '50_sma', '200_sma', '50_ema', 
                '100_ema', '12_ema', '26_ema', 'MACD_line', 'Signal_line', 
                'MACD_histogram', 'ADX', 'RSI', 'stoch_k', 'stoch_d', 'Williams_R',
                'ATR']


learning_rates = [0.1 ,0.01, 0.001]  
lookback_windows = [5, 7, 10, 15,20]
n_future = 1
target_col = -1         

initial_feature = ['Close'] # Starting with the closing price

# Get all combinations of the features list and add to the initial feature (Closing Price)
feature_combinations = []
for i in range(len(features) + 1):
    for combination in itertools.combinations(features, i):
        feature_combinations.append(list(combination)+ initial_feature )

print(f"Total number of combinations: {len(feature_combinations)}")

filepath = './Data/EURUSD_D1.csv'

multivariateDataX = multivariateFeatureEngineering(filepath) #Include addtional features outside of OHLCV
cols = [col for col in multivariateDataX.columns if col!='Close'] +['Close'] #shift Close to end
multivariateDataX =multivariateDataX[cols]
multivariateDataX = multivariateDataX.drop(columns=['Time']).reset_index(drop=True) #drop time column reset index


# Grid search loop
best_model = None
best_mae = 1 # Where the previous program crashed
with open(csv_file, mode='w', newline='') as file:
  writer = csv.writer(file)
    
  writer.writerow(header) # header

for lr in learning_rates:
    for n_past_window_size in lookback_windows:
        for features in feature_combinations:
            # update dataset to contain only features selected
            multivariateDataNew = multivariateDataX[list(features)]

            m_data = multivariateDataNew.to_numpy()

            # Create lagged features based on the window size
            m_X, m_Y = multivariateFeatureLag(m_data,n_past_window_size, n_future, target_col )

            # Train/test split
            X_train, X_test, y_train, y_test = train_test_split(m_X, m_Y, test_size=0.2, random_state=42, shuffle=False)

            # Scale features
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            # Fit model
            model = LinearRegression(lr, n_iterations=1000)
            model.fit(X_train, y_train)

            # Make predictions
            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test)

            # Check for NaN in predictions and handle them (e.g., replace with 0)
            y_pred_test = np.nan_to_num(y_pred_test)

            # Calculate metrics
            test_mse = mean_squared_error(y_test, y_pred_test)
            test_mae = mean_absolute_error(y_test, y_pred_test)
            test_mbe = np.mean(y_pred_test - y_test) # mean bias error
            test_r2 = r2_score(y_test, y_pred_test)
            # MAPE,

            # store improving models
            if test_mae < best_mae:
                best_mae = test_mae
                 # Log results to CSV
                with open(csv_file, mode='a', newline='') as file:
                  writer = csv.writer(file)
                  writer.writerow([lr, n_past_window_size, features, test_mse, test_mae,test_mbe, test_r2])  # Write the result row


# Save the best model (you can save the model using joblib or pickle)
print(f"Best model found with Test MSE: {best_mae}")





KeyboardInterrupt: 